In [1]:
import os

os.environ["OMP_NUM_THREADS"] = "8"
os.environ["MKL_NUM_THREADS"] = "8"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["NCCL_P2P_LEVEL"] = "PIX"
os.environ["MAX_JOBS"] = "16"

os.environ["CUDA_VISIBLE_DEVICES"] = "6"

import json
import random
import numpy as np
import torch
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    TextGenerationPipeline,
    LlamaTokenizer,
    PreTrainedTokenizer,
)

from fastchat.modules.gptq_utils.llm_dataset_adapter import get_dataset_adapter
from fastchat.train.data_modules.sft_dataset import load_sft_dataset, combine_dataset
from fastchat.train.train import LazySupervisedDataset
# from utils.llm_dataset_adapter import BaseDatasetAdapter
# from typing import List, Optional
# from datasets import load_dataset

/home/ados/anaconda3/envs/fastchat_train_2023dec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`AnnotionFormat` is deprecated and will be removed in v4.38. Please use `transformers.image_utils.AnnotationFormat` instead.


[2024-01-18 09:55:38,028] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/ados/anaconda3/envs/fastchat_train_2023dec/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [9]:
# load un-quantized model, by default, the model will always be loaded into CPU memory
tokenizer = AutoTokenizer.from_pretrained(
    model_config["model_path"], 
    local_files_only=True,
    model_max_length=model_config["max_length"],
)

tokenizer.pad_token = tokenizer.unk_token  # train_lora.py
tokenizer.padding_side = model_config["padding_side"]

raw_data = load_sft_dataset(model_config["data_path"])
dataset = LazySupervisedDataset(raw_data, tokenizer, model_config["data_format"])

choices = np.random.choice(range(len(dataset)), (model_config["n_samples"],), replace=False,).tolist()
examples = [
    {"input_ids": dataset[idx]["input_ids"], "attention_mask": dataset[idx]["attention_mask"]}
    for idx in choices
]

In [ ]:
tokenizer.decode(examples[0]['input_ids'])

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("/workspaces/disk0/data/llm_weights/MoMo-70B-lora-1.8.4-DPO/")
model = AutoModelForCausalLM.from_pretrained(
    "/data/llm_weights/merged/DIE-MoE-10.7Bx4_random",
    device_map='cpu',
)
model

/home/ados/anaconda3/envs/fastchat_train_2023dec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:10<00:00,  1.30s/it]


MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-47): 48 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=4096, out_features=4, bias=False)
          (experts): ModuleList(
            (0-3): 4 x MixtralBLockSparseTop2MLP(
              (w1): Linear(in_features=4096, out_features=14336, bias=False)
              (w2): Linear(in_features=14336, out_features=4096, bias=False)
              (w3): Linear(in_features=4096, out_features=14336, bias=

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "/data/llm_weights/custom_trained/DIE_F-10.7B_ep2/",
    device_map='cpu',
)
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
 

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "/workspaces/disk0/data/llm_weights/Platypus2-70B-instruct/",
    device_map='cpu',
)
model

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [03:24<00:00, 13.66s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 8192, padding_idx=0)
    (layers): ModuleList(
      (0-79): 80 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=8192, out_features=8192, bias=False)
          (k_proj): Linear(in_features=8192, out_features=1024, bias=False)
          (v_proj): Linear(in_features=8192, out_features=1024, bias=False)
          (o_proj): Linear(in_features=8192, out_features=8192, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=8192, out_features=28672, bias=False)
          (up_proj): Linear(in_features=8192, out_features=28672, bias=False)
          (down_proj): Linear(in_features=28672, out_features=8192, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
 

In [ ]:
text = "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nhi<|im_end|>\n<|im_start|>assistant\n"
inputs = tokenizer(text, return_tensors="pt")

outputs = model.generate(**inputs, max_new_tokens=1)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [3]:
# load quantized model to the first GPU
model = AutoGPTQForCausalLM.from_quantized("/workspaces/data/llm_weights/gptq/MoMo-70B-lora-1.8.4-DPO-GPTQ2", device="cuda:0")

Skipping module injection for FusedLlamaMLPForQuantizedModel as currently not supported with use_triton=False.


In [14]:
# inference with model.generate
user_input = "인공지능 분야의 LM이 무엇인가요?"
text = f"<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n{user_input}<|im_end|>\n<|im_start|>assistant\n"
# text = "auto_gptq is"
inputs = tokenizer(text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(outputs[0]))

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
인공지능 분야의 LM이 무엇인가요?<|im_end|>
<|im_start|>assistant
인공지능 분야의 LM은 "Language Model"의 약어입니다. Language Model은 자연어 처리 분야에서 사용되는 기술 중 하나로, 주어진 텍스트 데이터를 기반으로 다음 단어 또는 문장의 확률을 예측하는 모델입니다.


In [7]:
tokenizer.decode(outputs[0])

'auto_gptq is a Python package that provides a simple interface for generating'

In [ ]:
tokenizer

In [2]:
import time
def run_autogptq(model_config):
    quantize_config = BaseQuantizeConfig(
        bits=model_config["bits"],  # quantize model to 4-bit
        group_size=model_config[
            "group_size"
        ],  # it is recommended to set the value to 128
        desc_act=model_config[
            "desc_act"
        ],  # set to False can significantly speed up inference but the perplexity may slightly bad
        damp_percent=model_config["damp_percent"],
    )

    # load un-quantized model, by default, the model will always be loaded into CPU memory
    tokenizer = AutoTokenizer.from_pretrained(
        model_config["model_path"], 
        local_files_only=True,
        model_max_length=model_config["max_length"],
    )
    model = AutoGPTQForCausalLM.from_pretrained(
        model_config["model_path"],
        quantize_config,
        # use_safetensors=True,
        local_files_only=True,
        # device_map='auto',
        # max_memory={i: "80GIB" for i in range(torch.cuda.device_count())},
        
    )

    tokenizer.pad_token = tokenizer.unk_token  # train_lora.py
    tokenizer.padding_side = model_config["padding_side"]

    raw_data = load_sft_dataset(model_config["data_path"])
    dataset = LazySupervisedDataset(raw_data, tokenizer, model_config["data_format"])

    choices = np.random.choice(range(len(dataset)), (model_config["n_samples"],), replace=False,).tolist()
    examples = [
        {"input_ids": dataset[idx]["input_ids"], "attention_mask": dataset[idx]["attention_mask"]}
        for idx in choices
    ]
    
    start = time.time()
    model.quantize(examples, batch_size=4)
    print("quantization ellapse:", time.time() - start)
    # save quantized model using safetensors
    model.save_quantized(model_config["save_path"])  # , use_safetensors=True
    print(f"Successfully quantized at {model_config['save_path']}.")

In [4]:
%%time

model_config = {
    "model_path": "/workspaces/disk0/data/llm_weights/MoMo-70B-lora-1.8.4-DPO/",
    # "data_path": "/workspaces/data/llm_datasets/koalpaca/KoAlpaca_v1.1.jsonl",
    # "data_path": "/data/llm_datasets/custom/ados/sft/ados_msft_v4.json",
    "data_path": "/data/llm_datasets/custom/refined/wizardlm_orca_vicuna_dedup2.json",
    "save_path": "/workspaces/data/llm_weights/gptq/MoMo-70B-lora-1.8.4-DPO-GPTQ2",
    "data_format": "qwen",
    "padding_side": "right",
    "max_length": 4096,
    "bits": 4,
    "group_size": -1,
    "desc_act": True,
    "n_samples": 128,
    "damp_percent": 0.1,
}

run_autogptq(model_config)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:43<00:00,  1.44it/s]


quantization ellapse: 14300.248763084412
Successfully quantized at /workspaces/data/llm_weights/gptq/MoMo-70B-lora-1.8.4-DPO-GPTQ2.
CPU times: user 5h 18min 12s, sys: 1h 5min, total: 6h 23min 12s
Wall time: 4h 1s


In [5]:
########### automatically moving necessities into output folder 

import json
import os

with open(os.path.join(model_config["save_path"], "config.json"), "r") as json_file:
    data = json.load(json_file)

data["quantization_config"] = {
    "bits": model_config["bits"],
    "group_size": model_config["group_size"],
    "damp_percent": model_config["damp_percent"],
    "desc_act": model_config["desc_act"],
    "sym": True,
    "true_sequential": True,
    "model_name_or_path": None,
    "model_file_base_name": "model",
    "quant_method": "gptq",
}


with open(os.path.join(model_config["save_path"], "config.json"), "w") as json_file:
    json.dump(data, json_file, indent=2)
# vllm auto-gptq 적용하려면 config에 quantize_config 들어가야함
# 위 코드 오류 없으면 함수 내부에 넣기
# tokernizer 옮기는 코드 추가
import shutil

for f_path in [
    "special_tokens_map.json",
    "tokenizer_config.json",
    "tokenizer.json",
    "tokenizer.model",
]:
    src = os.path.join(model_config["model_path"], f_path)
    dst = os.path.join(model_config["save_path"], f_path)
    if os.path.exists(src):
        shutil.copy(src, dst)

# pad_token 2->0 으로 수정 "padding_side": "right" -> MingAI 0.5a에 적용

In [ ]:
%%time

model_config = {
    "model_path": "/workspaces/disk0/data/llm_weights/COKAL-ko-v1-70B",
    "data_path": "/workspaces/data/llm_datasets/koalpaca/KoAlpaca_v1.1.jsonl",
    "save_path": "/workspaces/data/llm_weights/gptq/COKAL-ko-v1-70B-GPTQ",
    "data_format": "orca",
    "padding_side": "right",
    "max_length": 4096,
    "bits": 4,
    "group_size": -1,
    "desc_act": True,
    "n_samples": 128,
    "damp_percent": 0.1,
}

run_autogptq(model_config)

In [ ]:
########### automatically moving necessities into output folder 

import json
import os

with open(os.path.join(model_config["save_path"], "config.json"), "r") as json_file:
    data = json.load(json_file)

data["quantization_config"] = {
    "bits": model_config["bits"],
    "group_size": model_config["group_size"],
    "damp_percent": model_config["damp_percent"],
    "desc_act": model_config["desc_act"],
    "sym": True,
    "true_sequential": True,
    "model_name_or_path": None,
    "model_file_base_name": "model",
    "quant_method": "gptq",
}


with open(os.path.join(model_config["save_path"], "config.json"), "w") as json_file:
    json.dump(data, json_file, indent=2)
# vllm auto-gptq 적용하려면 config에 quantize_config 들어가야함
# 위 코드 오류 없으면 함수 내부에 넣기
# tokernizer 옮기는 코드 추가
import shutil

for f_path in [
    "special_tokens_map.json",
    "tokenizer_config.json",
    "tokenizer.json",
    "tokenizer.model",
]:
    src = os.path.join(model_config["model_path"], f_path)
    dst = os.path.join(model_config["save_path"], f_path)
    if os.path.exists(src):
        shutil.copy(src, dst)

# pad_token 2->0 으로 수정 "padding_side": "right" -> MingAI 0.5a에 적용

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_config["model_path"], local_files_only=True
)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"
# dataset = get_dataset(model_config, tokenizer)

In [ ]:
text = " 안녕"
tokenizer(
    text,
    padding="max_length",
    max_length=4096,
)

In [ ]:
model = AutoGPTQForCausalLM.from_pretrained(
    model_config["model_path"],
    quantize_config,
    # use_safetensors=True,
    local_files_only=True,
    # device_map='auto',
)

In [ ]:
%%time

model_config = {
    "model_path": "/disk1/data/llm_weights/custom_trained/MingAI-70B-chat-orca_v0.5a-checkpoint-44362",
    "data_path": "/disk1/data/llm_datasets/koalpaca/KoAlpaca_v1.1.jsonl",
    "save_path": "/disk1/data/llm_weights/gptq/MingAI-70B-chat-orca_v0.5a-checkpoint-44362-GPTQ",
    "data_format": "orca",
    "max_length": 4096,
    "bits": 4,
    "group_size": -1,
    "desc_act": True,
    "n_samples": 128,
    "damp_percent": 0.1,
}

run_autogptq(model_config)

import json
import os

with open(os.path.join(model_config["save_path"], "config.json"), "r") as json_file:
    data = json.load(json_file)

data["quantization_config"] = {
    "bits": model_config["bits"],
    "group_size": model_config["group_size"],
    "damp_percent": model_config["damp_percent"],
    "desc_act": model_config["desc_act"],
    "sym": True,
    "true_sequential": True,
    "model_name_or_path": None,
    "model_file_base_name": "model",
    "quant_method": "gptq",
}


with open(os.path.join(model_config["save_path"], "config.json"), "w") as json_file:
    json.dump(data, json_file, indent=2)
# vllm auto-gptq 적용하려면 config에 quantize_config 들어가야함
# 위 코드 오류 없으면 함수 내부에 넣기
# tokernizer 옮기는 코드 추가
import shutil

for f_path in [
    "special_tokens_map.json",
    "tokenizer_config.json",
    "tokenizer.json",
    "tokenizer.model",
]:
    src = os.path.join(model_config["model_path"], f_path)
    dst = os.path.join(model_config["save_path"], f_path)
    if os.path.exists(src):
        shutil.copy(src, dst)

# llama 70b ko 기반모델 pad_token관련 파일 모두 수정.. 2->0 만약된다면..

In [ ]:
import json
import os

with open(
    os.path.join(
        "/disk1/data/llm_weights/gptq/Upstage-Llama-2-70B-instruct-v2-GPTQ",
        "config.json",
    ),
    "r",
) as json_file:
    data = json.load(json_file)

In [ ]:
data["quantization_config"] = {
    "bits": 4,
    "group_size": -1,
    "damp_percent": 0.01,
    "desc_act": True,
    "sym": True,
    "true_sequential": True,
    "model_name_or_path": None,
    "model_file_base_name": "model",
    "quant_method": "gptq",
}

In [ ]:
with open("config.json", "w") as json_file:
    json.dump(data, json_file, indent=2)

In [ ]:
%%time
model_config = {
    "model_path": "/disk1/data/llm_weights/custom_trained/MingAI-70B-chat-orca_v0.2_2/",
    "data_path": "/disk1/data/llm_datasets/koalpaca/KoAlpaca_v1.1.jsonl",
    "save_path": "/disk1/data/llm_weights/gptq/MingAI-70B-chat-orca_v0.2_2-GPTQ",
    "data_format": "orca",
    "max_length": 4096,
    "bits": 4,
    "group_size": -1,
    "desc_act": True,
    "n_samples": 128,
}

run_autogptq(model_config)

# model_config = {
#     'model_path': "/disk1/data/llm_weights/custom_trained/MingAI-70B-chat-orca_v0.2_2/",
#     'data_path': "/disk1/data/llm_datasets/custom/merged_korean_datasets-vicuna-v1.json",
#     'save_path': "/disk1/data/llm_weights/gptq/MingAI-70B-chat-orca_v0.2_2-GPTQ",
#     'data_format': "orca",
#     'max_length': 4096,
#     'bits': 4,
#     'group_size': 128,
#     'desc_act': False,
#     'n_samples': 128,
# }

# run_autogptq(model_config)

In [ ]:
%%time
model_config = {
    "model_path": "/disk1/data/llm_weights/custom_trained/MingAI-70B-chat-orca_v0.2_Llama2/",
    "data_path": "/disk1/data/llm_datasets/koalpaca/KoAlpaca_v1.1.jsonl",
    "save_path": "/disk1/data/llm_weights/gptq/MingAI-70B-chat-orca_v0.2_Llama2-GPTQ",
    "data_format": "orca",
    "max_length": 4096,
    "bits": 4,
    "group_size": -1,
    "desc_act": True,
    "n_samples": 128,
}

run_autogptq(model_config)